In [ ]:
import os
import numpy as np
import ceruleanml.data as data
from pathlib import Path
import pandas as pd
from icevision.parsers import COCOMaskParser
from icevision.data import SingleSplitSplitter

ml_data_path = os.path.join(os.path.abspath(os.getcwd()),"../../data/cv2_transfer/outputs/skytruth_annotation/first_phase/")
path = Path(ml_data_path)
class_folders = list(path.glob("*/"))

class_map = {
    "Infrastructure": 1,
    "Natural Seep": 2,
    "Coincident Vessel": 3,
    "Recent Vessel": 4,
    "Old Vessel": 5,
    "Ambiguous": 6,
    "Hard Negatives": 0,
}

parser = COCOMaskParser(annotations_filepath="../../data/cv2_transfer/instances_slicks_test_v2.json", img_dir="../tests/data/tiled_images")

train_records = parser.parse(data_splitter=SingleSplitSplitter())
record = train_records[0][0]

In [ ]:
from ceruleanml.coco_stats import region_props_for_instance_type
region_props_for_instance_type(train_records[0], "infra_slick")

In [ ]:
stats_df = pd.DataFrame(
    {'classes (integer)': class_list,
     'classes (string)': class_list_string,
     'avg. area of segment': avg_area_segment_list,
     'avg. area of bbox': avg_area_bbox_list,
     'avg. major axis length of segment': avg_major_axis_length_segment_list,
     'avg. minor axis length of segment': avg_minor_axis_length_segment_list,
     'std. dev. area of segment': stdv_area_segment_list,
     'std. dev. area of bbox': stdv_area_bbox_list,
     'std. dev. major axis length of segment': stdv_major_axis_length_segment_list,
     'std. dev. minor axis length of segment': stdv_minor_axis_length_segment_list
    })

In [ ]:
stats_df